<a href="https://colab.research.google.com/github/reachrkr/llamaindexrag/blob/main/LlamaIndex_Tutorials_VectorStore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q llama-index
!pip install -q openai
!pip install -q transformers
!pip install -q accelerate
!pip install -q optimum[exporters]
!pip install -q InstructorEmbedding
!pip install -q sentence_transformers
!pip install -q pypdf
!pip install -q chromadb

In [ ]:
!pip install -q llama-index chromadb --quiet
!pip install -q chromadb
!pip install -q sentence-transformers
!pip install -q pydantic==1.10.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.4 MB/s eta 0:00:00


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR-OPENAI-API-KEY"

In [ ]:
# !curl https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf --output IPCC_AR6_WGII_Chapter03.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20.7M  100 20.7M    0     0  79.7M      0 --:--:-- --:--:-- --:--:-- 79.7M


In [ ]:
# import
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.llms import OpenAI
from IPython.display import Markdown, display
import chromadb


In [ ]:
# define embedding function
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

In [ ]:
documents = SimpleDirectoryReader(
    input_files=["Orca_paper.pdf"]
).load_data()

# CHROMA-DB

In [ ]:
# create client and a new collection
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("orca_paper")

### Set up ChromaVectorStore and load in data


In [ ]:
# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)


In [ ]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
service_context = ServiceContext.from_defaults(embed_model=embed_model)

In [ ]:

index = VectorStoreIndex.from_documents(documents,
                                        storage_context=storage_context,
                                        service_context=service_context)

In [ ]:
# Query Data
query_engine = index.as_query_engine()

response = query_engine.query("What are some of the main contributions of this new Orca model?")

display(Markdown(f"{response}"))

The main contributions of the Orca model include its ability to retain the constraints and limitations of the LLaMA model family, its potential for efficient and scalable tool-augmented LFM systems, and its alignment with the GPT-4 model in terms of safety measures and content moderation. Additionally, the Orca model demonstrates relative safety in generating neutral content compared to other models like ChatGPT and GPT-4. It also showcases the potential for tool-augmented LFMs to address limitations and interact with the environment for up-to-date knowledge retrieval.

### How to Persist: Saving to Disk

In [ ]:
db = chromadb.PersistentClient(path="./chroma_db")


chroma_collection = db.get_or_create_collection("DB_collection")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

service_context = ServiceContext.from_defaults(embed_model=embed_model,
                                               chunk_size=800,
                                               chunk_overlap=20)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)

### Load from Disk

In [ ]:
# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")

chroma_collection = db2.get_or_create_collection("DB_collection")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
)

In [ ]:
# Query Data
query_engine = index.as_query_engine()

response = query_engine.query("What are some of the main contributions of this new Orca model?")

display(Markdown(f"{response}"))

Some nature-based solutions that can help us fight climate change include marine protected areas (MPAs), habitat restoration, habitat development, and maintaining sustainable fisheries. These solutions can protect ecosystem services, support biodiversity, and mitigate climate change. They can also enhance the resilience of marine ecosystems and provide societal and ecological benefits regardless of the level of climate change. Additionally, carefully designed and placed MPAs can increase resilience to climate change by removing additional stressors on ecosystems. Restoration of coastal habitats such as mangroves, salt marshes, and seagrass meadows can effectively remove carbon dioxide from the atmosphere and protect coasts from the impacts of storms and sea level rise. Sustainable fishing is also considered a nature-based solution as it contributes to food production and supports the UN's Sustainable Development Goal of Zero Hunger.

# PINECONE

In [ ]:
!pip install -q pinecone-client

In [ ]:
import pinecone

In [ ]:
os.environ['PINECONE_API_KEY'] = "PINECONE_API_KEY"
os.environ['PINECONE_ENVIRONMENT'] = 'gcp-starter'


In [ ]:
# initialize connection to pinecone
pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],
    environment=os.environ['PINECONE_ENVIRONMENT']
)

In [ ]:
# create the index if it does not exist already
index_name = 'llamaindex'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=768,
        metric='cosine'
    )

In [ ]:
pinecone_index = pinecone.Index("llamaindex")
pinecone.describe_index("llamaindex")

IndexDescription(name='llamaindex', metric='cosine', replicas=1, dimension=768.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [ ]:
from llama_index.vector_stores import PineconeVectorStore

In [ ]:
# set add_sparse_vector=True to compute sparse vectors during upsert
from llama_index.storage.storage_context import StorageContext

In [ ]:
vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index,
    # add_sparse_vector=True,
)


In [ ]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(embed_model=embed_model)


In [ ]:
index = VectorStoreIndex.from_documents(documents,
                                        storage_context=storage_context,
                                        service_context=service_context
                                        )

Upserted vectors:   0%|          | 0/55 [00:00<?, ?it/s]

In [ ]:
# Query Data
query_engine = index.as_query_engine()

response = query_engine.query("What is Big-Bench Hard test?")

display(Markdown(f"{response}"))

Big-Bench Hard is a suite of 23 challenging tasks that were introduced to measure the capabilities and limitations of large language models. These tasks are specifically designed to be difficult and have been selected based on the fact that prior language models did not outperform the average human-rater on them. The evaluation of Big-Bench Hard tasks is done using standard zero-shot prompting, without the use of any labeled examples.